In [1]:
import pynbody
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
galaxy_masses=pd.DataFrame()
BH_halos=pd.DataFrame()

In [3]:
def loader(path):
    '''returns snapshot and halo'''
    snap=pynbody.load(path)
    snap.physical_units()
    h=snap.halos()
    return snap,h

def BH_finder(snap):
    '''returns subsnap of black holes'''
    return snap.star[pynbody.filt.BandPass('tform','-15 Gyr','0 Gyr')]

def analysis(halo, view):
    '''center snap on halo's center of mass (angmom gives errors if there aren't enough pixels)'''
    if view==1:
        pynbody.analysis.angmom.faceon(halo)
    elif view==2:
        pynbody.analysis.angmom.sideon(halo)
    else:
        pynbody.analysis.halo.center(halo)
        
def plot_BH_pos(h, BH, grpid, attribute, snapshot, cmap='Greys', w=100, save=False, view=0):
    '''plots position of black holes. Send halo array(h), black hole array(BH), grpid, attribute, and snapshot you want to plot 
    Optional: width/preferred scope(w) (default=100), colormap(default='Greys'), save(default=False)'''
    
    pynbody.plot.image(h[grpid].s,qty=attribute,cmap=cmap, width=w, log=False)
    plt.plot(np.array((BH[np.where(BH['amiga.grp']==grpid)]['pos'])).T[0],np.array((BH[np.where(BH['amiga.grp']==grpid)]['pos'])).T[1],'r+')
    plt.xlim(-w/2,w/2)
    plt.ylim(-w/2,w/2)
    plt.title('%s_h%s_%s_w%s'%(snapshot, grpid, attribute, w))
    
    if save==True:
        plt.savefig('plots/%s/h%s/h148_h%s_%s_w%s.png'%(snapshot, grpid, grpid, attribute, w))
    plt.show()
    
def add_mass_columns(hgrpid, grpid, snap_id):
    '''returns row with gas, stellar, and total masses for one halo tracked through time'''
    df=pd.DataFrame([[hgrpid.g['mass'].sum(), hgrpid.s['mass'].sum(), hgrpid['mass'].sum()]], index=[snap_id], columns=['H[%d] Gas'%(grpid),'H[%d] Star'%(grpid),'H[%d] Total'%(grpid)])
    df.units='Msol'
    return df

def mass_df(h, BHgrp, snap_id):
    df=pd.DataFrame()
    for i in BHgrp:
        df=pd.concat([df, add_mass_columns(h[i], i, snap_id)], axis=1, join='outer')
    
    df.units='Msol'
    return df

def galaxy_df(snap_id_array,h,BHgrp):
    galaxy_masses=pd.DataFrame()
    for i in snap_id_array:
        vars()['mass_%s'%(i)]=mass_df(h,BHgrp,i)
        galaxy_masses=galaxy_masses.append(vars()['mass_%s'%(i)])

    return galaxy_masses

def plot_SFH(h, grpid,  snapshot, ls='dashed', c='k', save=True, mf=True):
    '''plots SFH. Send halo array(h[grpid]), linestyle(ls=dashed), color(c=k) grpid, and snapshot you want to plot 
    Optional:save(default=True)'''
    pynbody.plot.stars.sfh(h[grpid].s,linestyle=ls,color=c, massform=mf);
    plt.title('%s_h[%s]_SFH'%(snapshot,grpid), y=1.14);
    plt.savefig('plots/SFH/%s_h[%s]_SFH'%(snapshot, grpid));
    
    
def auto_plot_SFH(h, BHgrp,  snapshot, lines='dashed', col='k', sve=True, mform=True):
    '''plots SFH. Send halo array(h[grpid]), grpid, and snapshot you want to plot 
    Optional:linestyle(ls=dashed), color(c=k), save(default=True)'''
    for i in BHgrp:
        plot_SFH(h,i,snapshot, ls=lines,c=col, mf=mform, save=sve)

In [185]:
#array of last four digits for each snap
snap_id_array=['0139','0225','0275','0640','0974','1024','1269','1280','1408','1740','2048','2088','2432','2688','2816','2944','3072','3195','3200','3328','3456','3584','3606','3712','3840','3968','4096']

In [266]:
#snap id index (negative values start from end)
snap_id=snap_id-1
snap_id_array[snap_id]

'0139'

In [267]:
#set path
path='/data/scratch/jillian/h148/h148.cosmo50PLK.3072g3HbwK1BH.00%s/h148.cosmo50PLK.3072g3HbwK1BH.00%s'%(snap_id_array[snap_id],snap_id_array[snap_id])

In [268]:
#returns snapshot and halos in physical units (takes a couple of minutes)
snap,h=loader(path)

In [269]:
#returns subsnap of black holes
BH=BH_finder(snap)

In [270]:
df=pd.DataFrame({'%s halo groups'%(snap_id_array[snap_id]):BH['amiga.grp'],'%s iords'%(snap_id_array[snap_id]):BH['iord']})
BH_halos=pd.concat([BH_halos, df], axis=1, join='outer')

In [271]:
BH_halos

,4096 halo groups,4096 iords,3968 halo groups,3968 iords,3840 halo groups,3840 iords,3712 halo groups,3712 iords,3606 halo groups,3606 iords,3584 halo groups,3584 iords,3456 halo groups,3456 iords,3328 halo groups,3328 iords,3200 halo groups,3200 iords,3195 halo groups,3195 iords,3072 halo groups,3072 iords,2944 halo groups,2944 iords,2816 halo groups,2816 iords,2688 halo groups,2688 iords,2432 halo groups,2432 iords,2088 halo groups,2088 iords,2048 halo groups,2048 iords,1740 halo groups,1740 iords,1408 halo groups,1408 iords,1280 halo groups,1280 iords,1269 halo groups,1269 iords,1024 halo groups,1024 iords,0974 halo groups,0974 iords,0640 halo groups,0640 iords,0275 halo groups,0275 iords,0225 halo groups,0225 iords,0139 halo groups,0139 iords
0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1.0,101863510.0,1,101863510,1,101863510,5,101863510,4,101863510,5.0,101863510.0
1,3.0,101863565.0,3.0,101863565.0,3.0,101863565.0,3.0,101863565.0,3.0,101863565.0,3.0,101863565.0,3.0,101863565.0,3.0,101863565.0,3.0,101863565.0,3.0,101863565.0,3.0,101863565.0,4.0,101863565.0,4.0,101863565.0,4.0,101863565.0,4.0,101863565.0,4.0,101863565.0,4.0,101863565.0,3.0,101863565.0,3.0,101863565.0,4.0,101863565.0,4.0,101863565.0,4.0,101863565.0,6,101863565,13,101863565,10,101863565,8,101863565,3.0,101863565.0
2,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,1.0,101863726.0,67.0,101863726.0,67.0,101863726.0,42.0,101863726.0,6,101863705,13,101863705,10,101863705,8,101863705,3.0,101863705.0
3,1.0,101863732.0,1.0,101863732.0,1.0,101863732.0,1.0,101863732.0,1.0,101863732.0,1.0,101863732.0,1.0,101863732.0,1.0,101863732.0,1.0,101863732.0,1.0,101863732.0,1.0,101863732.0,1.0,101863732.0,1.0,101863732.0,4.0,101863727.0,4.0,101863727.0,4.0,101863727.0,4.0,101863727.0,3.0,101863727.0,3.0,101863727.0,4.0,101863727.0,4.0,101863727.0,23.0,101863727.0,1,101863726,35,101863726,27,101863726,11,101863726,10.0,101863726.0
4,649.0,101863734.0,631.0,101863734.0,523.0,101863734.0,504.0,101863734.0,503.0,101863734.0,503.0,101863734.0,499.0,101863734.0,484.0,101863734.0,469.0,101863734.0,469.0,101863734.0,488.0,101863734.0,319.0,101863734.0,310.0,101863734.0,1.0,101863732.0,1.0,101863732.0,1.0,101863732.0,1.0,101863732.0,1.0,101863732.0,24.0,101863732.0,1.0,101863732.0,1.0,101863732.0,11.0,101863732.0,22,101863727,22,101863727,7,101863727,5,101863727,8.0,101863727.0
5,6.0,101863739.0,6.0,101863739.0,6.0,101863739.0,6.0,101863739.0,6.0,101863739.0,6.0,101863739.0,6.0,101863739.0,5.0,101863739.0,5.0,101863739.0,5.0,101863739.0,5.0,101863739.0,5.0,101863739.0,5.0,101863739.0,312.0,101863734.0,324.0,101863734.0,325.0,101863734.0,312.0,101863734.0,516.0,101863734.0,109.0,101863734.0,93.0,101863734.0,90.0,101863734.0,97.0,101863734.0,11,101863732,8,101863732,21,101863732,23,101863732,33.0,101863732.0
6,11.0,101863883.0,11.0,101863883.0,10.0,101863883.0,10.0,101863883.0,10.0,101863883.0,10.0,101863883.0,11.0,101863883.0,11.0,101863883.0,10.0,101863883.0,10.0,101863883.0,8.0,101863883.0,10.0,101863883.0,10.0,101863883.0,5.0,101863739.0,5.0,101863739.0,6.0,101863739.0,6.0,101863739.0,6.0,101863739.0,6.0,101863739.0,7.0,101863739.0,7.0,101863739.0,10.0,101863739.0,95,101863734,141,101863734,337,101863734,302,101863734,183.0,101863734.0
7,12.0,101864796.0,12.0,101864796.0,12.0,101864796.0,12.0,101864796.0,12.0,101864796.0,12.0,101864796.0,13.0,101864796.0,13.0,101864796.0,13.0,101864796.0,13.0,101864796.0,12.0,101864796.0,14.0,101864796.0,15.0,101864796.0,10.0,101863

In [272]:
vars()['mass_%s'%(snap_id_array[snap_id])]=mass_df(h,np.unique(BH['amiga.grp']),snap_id_array[snap_id])

In [273]:
galaxy_masses

,H[109] Gas,H[109] Star,H[109] Total,H[10] Gas,H[10] Star,H[10] Total,H[115] Gas,H[115] Star,H[115] Total,H[11] Gas,H[11] Star,H[11] Total,H[12] Gas,H[12] Star,H[12] Total,H[130] Gas,H[130] Star,H[130] Total,H[131] Gas,H[131] Star,H[131] Total,H[13] Gas,H[13] Star,H[13] Total,H[141] Gas,H[141] Star,H[141] Total,H[145] Gas,H[145] Star,H[145] Total,H[149] Gas,H[149] Star,H[149] Total,H[14] Gas,H[14] Star,H[14] Total,H[157] Gas,H[157] Star,H[157] Total,H[158] Gas,H[158] Star,H[158] Total,H[15] Gas,H[15] Star,H[15] Total,H[160] Gas,H[160] Star,H[160] Total,H[16] Gas,H[16] Star,H[16] Total,H[170] Gas,H[170] Star,H[170] Total,H[17] Gas,H[17] Star,H[17] Total,H[19] Gas,H[19] Star,H[19] Total,H[1] Gas,H[1] Star,H[1] Total,H[20] Gas,H[20] Star,H[20] Total,H[21] Gas,H[21] Star,H[21] Total,H[22] Gas,H[22] Star,H[22] Total,H[23] Gas,H[23] Star,H[23] Total,H[24] Gas,H[24] Star,H[24] Total,H[25] Gas,H[25] Star,H[25] Total,H[26] Gas,H[26] Star,H[26] Total,H[27] Gas,H[27] Star,H[27] Total,H[2] Gas,H[2] Star,H[2] Total,H[302] Gas,H[302] Star,H[302] Total,H[310] Gas,H[310] Star,H[310] Total,H[312] Gas,H[312] Star,H[312] Total,H[319] Gas,H[319] Star,H[319] Total,H[31] Gas,H[31] Star,H[31] Total,H[324] Gas,H[324] Star,H[324] Total,H[325] Gas,H[325] Star,H[325] Total,H[32] Gas,H[32] Star,H[32] Total,H[337] Gas,H[337] Star,H[337] Total,H[35] Gas,H[35] Star,H[35] Total,H[36] Gas,H[36] Star,H[36] Total,H[3] Gas,H[3] Star,H[3] Total,H[40] Gas,H[40] Star,H[40] Total,H[41] Gas,H[41] Star,H[41] Total,H[42] Gas,H[42] Star,H[42] Total,H[44] Gas,H[44] Star,H[44] Total,H[45] Gas,H[45] Star,H[45] Total,H[469] Gas,H[469] Star,H[469] Total,H[47] Gas,H[47] Star,H[47] Total,H[484] Gas,H[484] Star,H[484] Total,H[488] Gas,H[488] Star,H[488] Total,H[48] Gas,H[48] Star,H[48] Total,H[499] Gas,H[499] Star,H[499] Total,H[4] Gas,H[4] Star,H[4] Total,H[503] Gas,H[503] Star,H[503] Total,H[504] Gas,H[504] Star,H[504] Total,H[516] Gas,H[516] Star,H[516] Total,H[523] Gas,H[523] Star,H[523] Total,H[57] Gas,H[57] Star,H[57] Total,H[5] Gas,H[5] Star,H[5] Total,H[631] Gas,H[631] Star,H[631] Total,H[63] Gas,H[63] Star,H[63] Total,H[649] Gas,H[649] Star,H[649] Total,H[65] Gas,H[65] Star,H[65] Total,H[67] Gas,H[67] Star,H[67] Total,H[6] Gas,H[6] Star,H[6] Total,H[70] Gas,H[70] Star,H[70] Total,H[7] Gas,H[7] Star,H[7] Total,H[8] Gas,H[8] Star,H[8] Total,H[90] Gas,H[90] Star,H[90] Total,H[93] Gas,H[93] Star,H[93] Total,H[95] Gas,H[95] Star,H[95] Total,H[97] Gas,H[97] Star,H[97] Total,H[9] Gas,H[9] Star,H[9] Total
4096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1139588130.39,135332344.929,10574577743.8,7503452.80862,104376733.617,9287775420.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.67460004583e+11,1.88270265251e+11,2.10912325919e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2983605593.03,1502495240.77,48580055355.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,984413313.128,431322510.739,33280094068.2,NaN,NaN,NaN,NaN,NaN,NaN,0.0,119724.141995,80225972.3471,NaN,NaN,NaN,NaN,NaN,NaN,1012776517.91,331596785.792,30584764500.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1193792825.96,128271939.268,11288723106.7,9882573.74956,104386453.11,9343441273.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.69230839461e+11,1.83605054266e+11,2.08632591288e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [274]:
galaxy_masses=galaxy_masses.append(vars()['mass_%s'%(snap_id_array[snap_id])])

In [275]:
galaxy_masses

,H[109] Gas,H[109] Star,H[109] Total,H[10] Gas,H[10] Star,H[10] Total,H[115] Gas,H[115] Star,H[115] Total,H[11] Gas,H[11] Star,H[11] Total,H[12] Gas,H[12] Star,H[12] Total,H[130] Gas,H[130] Star,H[130] Total,H[131] Gas,H[131] Star,H[131] Total,H[13] Gas,H[13] Star,H[13] Total,H[141] Gas,H[141] Star,H[141] Total,H[145] Gas,H[145] Star,H[145] Total,H[149] Gas,H[149] Star,H[149] Total,H[14] Gas,H[14] Star,H[14] Total,H[157] Gas,H[157] Star,H[157] Total,H[158] Gas,H[158] Star,H[158] Total,H[15] Gas,H[15] Star,H[15] Total,H[160] Gas,H[160] Star,H[160] Total,H[16] Gas,H[16] Star,H[16] Total,H[170] Gas,H[170] Star,H[170] Total,H[17] Gas,H[17] Star,H[17] Total,H[183] Gas,H[183] Star,H[183] Total,H[19] Gas,H[19] Star,H[19] Total,H[1] Gas,H[1] Star,H[1] Total,H[20] Gas,H[20] Star,H[20] Total,H[21] Gas,H[21] Star,H[21] Total,H[22] Gas,H[22] Star,H[22] Total,H[23] Gas,H[23] Star,H[23] Total,H[24] Gas,H[24] Star,H[24] Total,H[25] Gas,H[25] Star,H[25] Total,H[26] Gas,H[26] Star,H[26] Total,H[27] Gas,H[27] Star,H[27] Total,H[2] Gas,H[2] Star,H[2] Total,H[302] Gas,H[302] Star,H[302] Total,H[310] Gas,H[310] Star,H[310] Total,H[312] Gas,H[312] Star,H[312] Total,H[319] Gas,H[319] Star,H[319] Total,H[31] Gas,H[31] Star,H[31] Total,H[324] Gas,H[324] Star,H[324] Total,H[325] Gas,H[325] Star,H[325] Total,H[32] Gas,H[32] Star,H[32] Total,H[337] Gas,H[337] Star,H[337] Total,H[33] Gas,H[33] Star,H[33] Total,H[35] Gas,H[35] Star,H[35] Total,H[36] Gas,H[36] Star,H[36] Total,H[3] Gas,H[3] Star,H[3] Total,H[40] Gas,H[40] Star,H[40] Total,H[41] Gas,H[41] Star,H[41] Total,H[42] Gas,H[42] Star,H[42] Total,H[44] Gas,H[44] Star,H[44] Total,H[45] Gas,H[45] Star,H[45] Total,H[469] Gas,H[469] Star,H[469] Total,H[47] Gas,H[47] Star,H[47] Total,H[484] Gas,H[484] Star,H[484] Total,H[488] Gas,H[488] Star,H[488] Total,H[48] Gas,H[48] Star,H[48] Total,H[499] Gas,H[499] Star,H[499] Total,H[4] Gas,H[4] Star,H[4] Total,H[503] Gas,H[503] Star,H[503] Total,H[504] Gas,H[504] Star,H[504] Total,H[516] Gas,H[516] Star,H[516] Total,H[523] Gas,H[523] Star,H[523] Total,H[53] Gas,H[53] Star,H[53] Total,H[57] Gas,H[57] Star,H[57] Total,H[5] Gas,H[5] Star,H[5] Total,H[631] Gas,H[631] Star,H[631] Total,H[63] Gas,H[63] Star,H[63] Total,H[649] Gas,H[649] Star,H[649] Total,H[65] Gas,H[65] Star,H[65] Total,H[67] Gas,H[67] Star,H[67] Total,H[6] Gas,H[6] Star,H[6] Total,H[70] Gas,H[70] Star,H[70] Total,H[7] Gas,H[7] Star,H[7] Total,H[8] Gas,H[8] Star,H[8] Total,H[90] Gas,H[90] Star,H[90] Total,H[93] Gas,H[93] Star,H[93] Total,H[95] Gas,H[95] Star,H[95] Total,H[97] Gas,H[97] Star,H[97] Total,H[9] Gas,H[9] Star,H[9] Total
4096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1139588130.39,135332344.929,10574577743.8,7503452.80862,104376733.617,9287775420.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.67460004583e+11,1.88270265251e+11,2.10912325919e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2983605593.03,1502495240.77,48580055355.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,984413313.128,431322510.739,33280094068.2,NaN,NaN,NaN,NaN,NaN,NaN,0.0,119724.141995,80225972.3471,NaN,NaN,NaN,NaN,NaN,NaN,1012776517.91,331596785.792,30584764500.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1193792825.96,128271939.268,11288723106.7,9882573.74956,104386453.11,9343441273.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

#Use when plotting to center and adjust angle of perspective: view=1 for face on and 2 for side on, anything else will leave it as it is
view=0
analysis(h[grpid],view)

#plot location of black holes
plot_BH_pos(h, BH_test, grpid, attribute='tform', snapshot=snap_id_array[snap_id])

In [277]:
BH_halos.to_csv('ALL_BH_iords_grps.csv')

In [278]:
galaxy_masses.to_csv('ALL_galaxy_masses.csv')